# Code for loading data, setting up a model, running training and making predictions

Copied from `testEDA_pontus.ipynb`, only including the relevant parts for the project solution. Does not include tests or unused functions.
Also made some parts more generalized, for example making it possible to run the model for arbitrary batch size.

Implement new ideas and tweak parameters using this notebook, try to improve the results. Save checkpoints/good models as separate notebooks.

In [1]:
# Import packages
import json
import pandas as pd  
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch.nn import Linear
from torch.nn import ReLU
import torch.nn.functional as F

## Load data

In [31]:
# Read data
train = pd.read_json('data/train.json', lines=True) 
test = pd.read_json('data/test.json', lines=True) 
# Divide test data into the two subsets: Private Test and Public Test
# seq_length=107 in Public Test while seq_length=130 in Private Test
test_public = test[test["seq_length"] == 107]
test_private = test[test["seq_length"] == 130]

# Print the first sample for testing
df = pd.DataFrame(train)
print(df.iloc[0])

# Optionally, only take training data which have passed the signal-to-noise filter
train_filtered = train[train["SN_filter"] == 1]

# Change apply_SN_filter to True to only train on filtered data, using the SN filter described in the Kaggle challenge (same which is used for public test data)
apply_SN_filter = True
# apply_SN_filter = False 
if apply_SN_filter == True:
    train = train_filtered

index                                                                  0
id                                                          id_001f94081
sequence               GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...
structure              .....((((((.......)))).)).((.....((..((((((......
predicted_loop_type    EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...
signal_to_noise                                                    6.894
SN_filter                                                              1
seq_length                                                           107
seq_scored                                                            68
reactivity_error       [0.1359, 0.20700000000000002, 0.1633, 0.1452, ...
deg_error_Mg_pH10      [0.26130000000000003, 0.38420000000000004, 0.1...
deg_error_pH10         [0.2631, 0.28600000000000003, 0.0964, 0.1574, ...
deg_error_Mg_50C       [0.1501, 0.275, 0.0947, 0.18660000000000002, 0...
deg_error_50C          [0.2167, 0.34750000000000003

## Structure adjacency matrix

In [32]:
def get_struct_adj(data = train, sequential_edges = False):
    # Get adjacency matrix from sample structure sequence
    # Include edges between base pairs
    # If sequential_edges == False, do not include edges between sequential bases
    # If sequential_edges == True, add these edges, which correspond to the diagonals -1 and 1 in the adjacency matrix (assuming undirected edges)
    struct_adj = []
    for ix in range(len(data)):
        seq_length = data["seq_length"].iloc[ix]
        structure = data["structure"].iloc[ix]
        sequence = data["sequence"].iloc[ix]

        queue = [] # Store indices corresponding to "(" in queue

        sample_struct_adj = np.zeros([seq_length, seq_length])
        for jx in range(seq_length):
            if structure[jx] == "(":
                queue.append(jx) # Append index of "(" in base pair to queue
            elif structure[jx] == ")":
                start = queue.pop() # Retrieve index of last "(" in queue, corresponding to ")" at jx
                sample_struct_adj[start, jx] = 1 # Add edge from "(" to ")"
                sample_struct_adj[jx, start] = 1 # Add edge from ")" to "(" (assume undirected)

        if sequential_edges == True:
            ones = np.ones(seq_length-1) # Match length of -1 and 1 diagonals in sample_struct_adj
            sample_struct_adj += np.diag(ones,1) # Add sequential edges (i,i+1) 
            sample_struct_adj += np.diag(ones,-1) # Add sequential edges (i+1,i) (assume non-directed)

        struct_adj.append(sample_struct_adj)

    struct_adj = np.array(struct_adj)
    return struct_adj 

## Distance adjacency matrix

In [33]:
# Function for constructing distance adjacency matrix
# Only returns one distance adjacency matrix, since it is identical for all samples (only depends on number of nodes)
def get_dist_adj(data = train, power = 1):
    # Get adjacency matrix from inverse index-based distance between nodes
    # power is the variable p in the expression D(i,j)
    dist_adj = []
    idx = np.arange(data["seq_length"].iloc[0]) # Get number of nodes
    for ix in range(len(idx)):
        d = np.abs(idx[ix] - idx) # Get distance from individual nodes to all other nodes
        dist_adj.append(d)

    # Convert distance to distance measure according to formula    
    dist_adj = np.array(dist_adj) + 1 # Add one to avoid singularity at d=0
    dist_adj = 1/dist_adj # Inverse of distance
    dist_adj = dist_adj**power # Apply the specified power
    return dist_adj 

## Base pair probabilities

In [34]:
# Load the provided base pair probability adjacency matrices for the samples included in the datasets

# Train
Adj_bpps = []
for id in tqdm(train["id"]):
    bpps = np.load(f"data/bpps/{id}.npy")
    Adj_bpps.append(bpps)
Adj_bpps = np.array(Adj_bpps)

# Public test
Adj_bpps_test_public = []
for id in tqdm(test_public["id"]):
    bpps = np.load(f"data/bpps/{id}.npy")
    Adj_bpps_test_public.append(bpps)
Adj_bpps_test_public = np.array(Adj_bpps_test_public)

  0%|          | 0/1589 [00:00<?, ?it/s]

  0%|          | 0/629 [00:00<?, ?it/s]

## Node features

In [35]:
def get_node_features(data = train):
    # Create a node feature matrix for each sample in data
    # Encode feature vectors as one-hot arrays  
    # Included features: 
    #   Base (given by sequence)
    #   Loop type (given by predicted_loop_type)
    # Could also include sequence, i.e. "." "(" and ")", but I don't see how this provides any interesting information if the structure adjacency matrix is used
    X = [] # Stacked node feature matrices for all samples in data
    
    for ix in range(len(data)):
        seq_length = data["seq_length"].iloc[ix]
        sequence = data["sequence"].iloc[ix]
        predicted_loop_type = data["predicted_loop_type"].iloc[ix]

        X_sample = [] # Node feature matrix for current sample

        for jx in range(seq_length):
            # Base one hot
            bases = np.array(['A', 'G', 'U', 'C']) # Different order than reference notebook (A,G,C,U)
            x_base = np.zeros(len(bases))
            x_base[bases == sequence[jx]] = 1 # Set base one-hot to 1 at correct index

            # Predicted Loop Type one hot
            loop_types = np.array(['S', 'M', 'I', 'B', 'H', 'E', 'X'])
            x_loop = np.zeros(len(loop_types))
            x_loop[loop_types == predicted_loop_type[jx]] = 1 # Set loop-type one-hot to 1 at correct index

            x = np.concatenate((x_base,x_loop)) # Concatenate to one node feature vector
            X_sample.append(x) # Append node feature vector to node feature matrix
        X_sample = np.array(X_sample)
        X.append(X_sample) # Append node feature matrix for current graph
    X = np.array(X)
    return X

## Construct node features, adjacency matrix and targets for training data
Currently includes Structure and Distance adjacency matrices

In [36]:
# Construct node features and adjacency matrix for training data
print("Shapes of inputs - Train")

# Node features
X = get_node_features(data = train)
X = X.astype(np.float32) # Convert to floats to prepare for torch model
print("Node features X: (n_samples, n_nodes, n_node_features) ", X.shape)
# Structure adjacency 
Adj_pairs = get_struct_adj(data = train)
print("Structure adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_pairs.shape)
# Distance adjacency
Adj_dist = get_dist_adj(data = train, power = 1)
Adj_dist = Adj_dist[None, :,:] # Expand the dimensions of the array to allow stacking matrices for all samples 
Adj_dist = np.repeat(Adj_dist, len(train), axis = 0) # Repeat the distance array for each sample (they are identical, simply to match the data shape)
print("Distance adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_dist.shape)
# Base pair probability adjacency
print("Base pair probability adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_bpps.shape)
# Concatenate adjacency matrices into one array along last dimension
Adj = np.concatenate([Adj_pairs[:,:,:,None], Adj_dist[:,:,:,None], Adj_bpps[:,:,:,None]], axis = 3) # Expand dimensions of adjacency matrices and stack along new dimension
Adj = Adj.astype(np.float32) # Convert to floats to prepare for torch model
print("Total adjacency matrix: (n_samples, n_nodes, n_nodes, n_edge_features) ", Adj.shape)

Shapes of inputs - Train
Node features X: (n_samples, n_nodes, n_node_features)  (1589, 107, 11)
Structure adjacency matrices: (n_samples, n_nodes, n_nodes)  (1589, 107, 107)
Distance adjacency matrices: (n_samples, n_nodes, n_nodes)  (1589, 107, 107)
Base pair probability adjacency matrices: (n_samples, n_nodes, n_nodes)  (1589, 107, 107)
Total adjacency matrix: (n_samples, n_nodes, n_nodes, n_edge_features)  (1589, 107, 107, 3)


In [37]:
# Construct target arrays for training data
target_labels = ["reactivity", "deg_Mg_pH10", "deg_Mg_50C", "deg_pH10", "deg_50C"]

y_train = []
seq_length = train["seq_length"].iloc[0] # Get number of nodes (lenght of sequence)
seq_scored = train["seq_scored"].iloc[0] # Get number of nodes with ground truth targets
for target in target_labels:
    y = np.vstack(train[target]) # Create (n_samples, seq_scored) arrays for each target
    y_train.append(y) # Append array for each target
y_train = np.stack(y_train, axis=2) # Join the target arrays along last axis to match shape of feature arrays
y_train = y_train.astype(np.float32) # Convert to floats to prepare for torch model
print("Shape of targets: ", y_train.shape)


Shape of targets:  (1589, 68, 5)


## Define graph convolution layer

In [38]:
class myGraphConv(torch.nn.Module):
    """
    The graph neural network operator from the “Weisfeiler and Leman Go 
    Neural: Higher-order Graph Neural Networks” paper

    x' = x_i W_1.T + (Adj x_i) W_2.T

    Arguments:
        in_channels (int): Number of features (size) of each input node
        out_channels (int): Number of features (size) of each output node
        n_edge_features (int): Number of edge features, i.e. Adj.shape[-1]
    
    forward performs the graph neural network operation
    Arguments:
        x (torch tensor): The input node features of shape (n_samples, n_nodes, in_channels) 
        Adj (torch tensor): The adjacency matrix of the graph of shape (n_samples, n_nodes, n_nodes, n_edge_features)
    Returns: 
        x' (torch tensor): Output node feature matrix of shape (n_samples, n_nodes, out_channels)
    """
    # Notes:
    ## Also should add boolean argument bias for the linear weights.
    ## GraphConv seems to only use bias for W_2 if I understand the source code correctly
    ## (see lin_r = ... bias=False).a


    def __init__(self, in_channels, out_channels, n_edge_features):
        super(myGraphConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.n_edge_features = n_edge_features # Get number of edge features (number of stacked adjacency matrices)

        self.lin_self = Linear(in_channels, out_channels, bias=True) # bias=False to match GraphConv? Check source code
        if self.n_edge_features >= 1:
            self.lin_1 = Linear(in_channels, out_channels, bias=True)  
        if self.n_edge_features >= 2:
            self.lin_2 = Linear(in_channels, out_channels, bias=True)
        if self.n_edge_features >= 3:
            self.lin_3 = Linear(in_channels, out_channels, bias=True)
        if self.n_edge_features >= 4:
            raise ValueError("Number of edge features can not be larger than 3") # "Hard code" up to 3 edge features

        self.reset_parameters()
   
    def reset_parameters(self):
        self.lin_self.reset_parameters()
        if self.n_edge_features >= 1:
            self.lin_1.reset_parameters()
        if self.n_edge_features >= 2:
            self.lin_2.reset_parameters()
        if self.n_edge_features >= 3:
            self.lin_3.reset_parameters()
        
        

    def forward(self, x, Adj):
        # Shapes of arguments, weight matrices and output
        # x: (n_samples, n_nodes, in_channels) 
        # Adj: (n_samples, n_nodes, n_nodes, n_edge_features)
        # W_1: (in_channels, out_channels)
        # W_2: (in_channels, out_channels)
        # out: (n_samples, n_nodes, out_channels)

        # Confirm that the input variable n_edge_features matches the adjacency matrix
        if self.n_edge_features != Adj.shape[-1]:
            raise ValueError("Specified number of edge features must match last dimensino in adjacency matrix") 

        # Calculate contribution from self (node)
        out = self.lin_self(x)

        # Add contributions from edges
        # Calculate contributions from adjacent nodes
        # Use separate weights for each edge feature
        if self.n_edge_features >= 1:
            out_1 = torch.matmul(Adj[..., 0], x) # This is equivalent to summing over edge weights assuming Adj contains the edge weights
            out_1 = self.lin_1(out_1) # Multiply with weight matrix for adjacent nodes
            out += out_1 # Add contribution from first edge feature
        # Repeat for all edge weights
        if self.n_edge_features >= 2:
            out_2 = torch.matmul(Adj[..., 1], x) 
            out_2 = self.lin_2(out_2) 
            out += out_2 # Add contribution from second edge feature
        if self.n_edge_features >= 3:
            out_3 = torch.matmul(Adj[..., 2], x)
            out_3 = self.lin_3(out_3) 
            out += out_3 # Add contribution from third edge feature
        return out

    # The method that returns a printable representation of the operator, copy to match GraphConv source code 
    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)


## Define loss function

In [39]:
# Define MCRMSE loss function
# Include all 5 targets by default, allow optional argument to calculate MCRMSE of scored targets only.
# Assumes targets are ordered such that the first 3 targets are the scored ones.
# Inputs should have dimensions (n_samples, n_nodes, n_targets)
def MCRMSE(y_true, y_pred, only_scored=False, data = train):
    # Reshape if input only includes one sample and has dimensions (n_nodes, n_targets)
    if y_true.dim() == 2:
        y_true = y_true[None, :, :]
    if y_pred.dim() == 2:
        y_pred = y_pred[None, :, :]

    # Extract the scored targets
    seq_scored = data["seq_scored"].iloc[0] # Get number of nodes with ground truth targets
    y_pred = y_pred[:, :seq_scored, :] 
    # true = y_true[:, :seq_scored, :] # Not necessary since only scored targets are included, could include dummy values instead as in reference notebook

    y_diff = y_pred - y_true
    mse = torch.mean(y_diff**2, axis=1) # Average over nodes in each sample for every target
    rmse = torch.sqrt(mse)
    
    num_scored = 5 # Include all targets by default
    if only_scored == True:
        num_scored = 3 # Include only scored targets if specified by keyword (assumes correct ordering of targets in y_true and y_pred)

    mcrmse = torch.mean(rmse[:, :num_scored], axis=1) # Average over included targets

    return mcrmse

## Define model

In [40]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, n_edge_features, n_node_features = 11):
        super(GNN, self).__init__()
        torch.manual_seed(12345) # For reproducible results
        self.conv = myGraphConv(n_node_features, hidden_channels, n_edge_features)
        self.lin = Linear(hidden_channels, 5) # Map to the 5 output targets with dense layer
        self.relu = ReLU()

    def forward(self, x, Adj):
        # 1. Obtain node embeddings, use GraphConv layers with ReLU for non-linearity
        x = self.conv(x, Adj) # Give adjacency matrix instead of edge_index and edge_weight
        x = self.relu(x)

        # 2. Readout layer
        # No pooling is required, we want target labels for each node, not for the entire graph

        # 3. Apply a final classifier 
        # Use a single layer as classifier to map to the targets
        x = self.lin(x)

        # No LogSoftmax needed, possibly some other function to map to correct targets?

        return x
    

## Train model

In [41]:
# Instantiate GNN model, optimizer and loss function
model = GNN(hidden_channels=64, n_edge_features = Adj.shape[-1])
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # Adjust learning rate
criterion = MCRMSE # Mean column-wise root mean square error (MCRMSE) loss

# Define trainer function for GNN
def run_training(X_data, Adj_data):
    model.train()
    running_loss = 0 # For printing training loss
    for ix in range(len(X_data)):  # Iterate over samples in the training dataset
        out = model(X_data[ix], Adj_data[ix]) # Perform a single forward pass.
    # out = model(X_data, Adj_data) # all in one pass, no for loop
        loss = criterion(torch.tensor(y_train[ix]), out)  # Compute the loss
        loss.backward()  # Derive gradients
        optimizer.step()  # Update parameters based on gradients
        optimizer.zero_grad()  # Clear gradients

        # Print statistics
        running_loss += loss.item()
        if ix % 200 == 199:    # Print average loss every 200 mini-batches (every 200 samples in this case)
            print('[sample %5d] loss: %.3f' %
                    (ix + 1, running_loss / 200))
            running_loss = 0.0 # Reset running loss

# Convert training data inputs to pytorch tensors and run training
X_torch = torch.tensor(X)
Adj_torch = torch.tensor(Adj)
run_training(X_torch, Adj_torch)


GNN(
  (conv): myGraphConv(11, 64)
  (lin): Linear(in_features=64, out_features=5, bias=True)
  (relu): ReLU()
)
[sample   200] loss: 0.446
[sample   400] loss: 0.409
[sample   600] loss: 0.393
[sample   800] loss: 0.390
[sample  1000] loss: 0.370
[sample  1200] loss: 0.383
[sample  1400] loss: 0.383


## Construct node features and adjacency matrix for test data

In [42]:
# Construct node features and adjacency matrix for test data
print("Shapes of inputs - Public test")

# Node features
X_test_public = get_node_features(data = test_public)
X_test_public = X_test_public.astype(np.float32) # Convert to floats to prepare for torch model
print("Node features X: (n_samples, n_nodes, n_node_features) ", X_test_public.shape)
# Structure adjacency 
Adj_pairs_test_public = get_struct_adj(data = test_public)
print("Structure adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_pairs_test_public.shape)
# Distance adjacency
Adj_dist_test_public = get_dist_adj(data = test_public, power = 1)
Adj_dist_test_public = Adj_dist_test_public[None, :,:] # Expand the dimensions of the array to allow stacking matrices for all samples 
Adj_dist_test_public = np.repeat(Adj_dist_test_public, len(test_public), axis = 0) # Repeat the distance array for each sample (they are identical, simply to match the data shape)
print("Distance adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_dist_test_public.shape)
# Base pair probability adjacency
print("Base pair probability adjacency matrices: (n_samples, n_nodes, n_nodes) ", Adj_bpps_test_public.shape)
# Concatenate adjacency matrices into one array along last dimension
Adj_test_public = np.concatenate([Adj_pairs_test_public[:,:,:,None], Adj_dist_test_public[:,:,:,None], Adj_bpps_test_public[:,:,:,None]], axis = 3) # Expand dimensions of adjacency matrices and stack along new dimension
Adj_test_public = Adj_test_public.astype(np.float32) # Convert to floats to prepare for torch model
print("Total adjacency matrix: (n_samples, n_nodes, n_nodes, n_edge_features) ", Adj_test_public.shape)

Shapes of inputs - Public test
Node features X: (n_samples, n_nodes, n_node_features)  (629, 107, 11)
Structure adjacency matrices: (n_samples, n_nodes, n_nodes)  (629, 107, 107)
Distance adjacency matrices: (n_samples, n_nodes, n_nodes)  (629, 107, 107)
Base pair probability adjacency matrices: (n_samples, n_nodes, n_nodes)  (629, 107, 107)
Total adjacency matrix: (n_samples, n_nodes, n_nodes, n_edge_features)  (629, 107, 107, 3)


## Make predictions on public test data

In [43]:
# Define prediction function
def run_prediction(X_data, Adj_data):
    model.eval()
    y_pred = []
    outs = model(X_data, Adj_data) # Feed the data through the network 
    for yx in outs:
        y_pred.append(yx.detach().numpy())
    y_pred = np.array(y_pred)
    return y_pred
# Convert test data inputs to pytorch tensors and run prediction
X_test_public_torch = torch.tensor(X_test_public)
Adj_test_public_torch = torch.tensor(Adj_test_public)
y_pred = run_prediction(X_test_public_torch, Adj_test_public_torch)
print(y_pred.shape)

(629, 107, 5)


## Make predictions on training data using trained model

In [44]:
# Run prediction on training data as a test run
y_train_pred = run_prediction(X_torch, Adj_torch)
y_train_pred = y_train_pred.astype(np.float32)

# Calculate score on training data
y_train_torch = torch.tensor(y_train)
y_train_pred_torch = torch.tensor(y_train_pred)
training_score = MCRMSE(y_train_torch, y_train_pred_torch, only_scored=False)
training_score_only_scored = MCRMSE(y_train_torch, y_train_pred_torch, only_scored=True)

print(f"Mean score on training data, all 5 targets: {float(torch.mean(training_score)):.5}")
print(f"Mean score on training data, only scored targets: {float(torch.mean(training_score_only_scored)):.5}")

Mean score on training data, all 5 targets: 0.38034
Mean score on training data, only scored targets: 0.39525


To do in this notebook:
* Modify GraphConv and GNN so that the adjacency matrix Adj can have shape `[n_samples, n_nodes, n_nodes, n_edge_features]` *Done*
* Test the possible ways to include multiple edge features to see which seems to give the best results
* Modify training so that one can iterate over mini-batches *Done*
* Test if training over mini-batches helps with score

New in this notebook/currently implemented:
* Concatenation of adjacency matrices into one array with shape `[n_samples, n_nodes, n_nodes, n_edge_features]`
* myGraphConv that takes adjacency matrix with shape `[n_samples, n_nodes, n_nodes, n_edge_features]`, currently using separate term and weights for each edge feature, hard coded for up to 3 edge features
* Updated MCRMSE so that it can take both single samples and multiple at once, both as shapes [n_nodes,n_targets=5] and [n_samples, n_nodes,n_targets=5]
* Updated GNN and myGraphConv so that they can take both single samples and multiple at once
* Updated run_prediction to make predictions for all samples in one pass (no iteration required)
* Added bpps adjacency matrices, easy to include now with updates to GNN. Scores improved slightly.
* Added alternative GraphConv layer implementation, where the outputs of each adjacency matrix*x*W is stacked instead of added together. Added an additional linear layer to map to outputs. This improved the predictions a bit. However, this could be overfitting, since we are using more parameters when stacking the outputs from the GraphConv layer than when summing them. Moved this function to `testEDA_pontus.ipynb` to avoid clutter (myGraphConv_expand function and one line of code in GNN to call the right function).

Note that most functions are different than the ones in `testEDA_pontus.ipynb` with the same names, save as new functions if copied.